In [36]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [37]:
import nltk

# # This will download stopwords if not already present
# nltk.download('stopwords') wordNet

nltk.download('stopwords', download_dir='F:/Programming_Practise/MLOPs_Project1/notebooks')
nltk.download('wordnet', download_dir='F:/Programming_Practise/MLOPs_Project1/notebooks')
import nltk.data
nltk.data.path.append('F:/Programming_Practise/MLOPs_Project1/notebooks')
# nltk.data.path.append('F:/Programming_Practise/MLOPs_Project1/notebooks')



[nltk_data] Downloading package stopwords to
[nltk_data]     F:/Programming_Practise/MLOPs_Project1/notebooks...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     F:/Programming_Practise/MLOPs_Project1/notebooks...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
df=pd.read_csv("IMDB.csv")
df=df.sample(500)
df.to_csv("data.csv",index=False)
df.head()

,review,sentiment
323,A hilarious and insightful perspective of the ...,positive
439,The Story line for this game is very jerky whe...,positive
914,This is probably the best cinematic depiction ...,positive
903,"I saw ""Sweeney Todd"" on Broadway in 1980. It s...",positive
259,I am a massive fan of the book and Orwell is c...,negative


In [39]:
# Data Preprocessing

# Define Text preprocessing functions
def lemmatization(text):
    """ Lemmatize the text. """
    lemmatizer=WordNetLemmatizer()
    text=text.split()
    text=[lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """ Remove stopswords from the text """
    stop_words= set(stopwords.words("english"))
    text=[word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """ Remove numbers from the text """
    text= "".join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """ Convert text to lower case """
    text= text.split()
    text=[word.lower() for word in text]
    return " ".join(text)

def removing_punctations(text):
    """ Removing puncatations from the text """
    text= re.sub('[%s]' % re.escape(string.punctuation),' ', text)
    text=text.replace(':','')
    text=re.sub('\s+',' ',text).strip()
    return text

def removing_urls(text):
    """ Remove URLs from the text. """
    url_pattern=re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)

def normalize_text(df):
    """ Normalize the text data."""
    try: 
        df["review"]= df["review"].apply(lower_case)
        df["review"]= df["review"].apply(remove_stop_words)
        df["review"]= df["review"].apply(removing_numbers)
        df["review"]= df["review"].apply(removing_punctations)
        df["review"]= df["review"].apply(removing_urls)
        df["review"]= df["review"].apply(lemmatization)
        return df
    except Exception as e:
        print(f"Error during text normalization: {e}")
        raise

In [40]:
df = normalize_text(df)
df.head()

,review,sentiment
323,hilarious insightful perspective dating world ...,positive
439,story line game jerky appears game writer trie...,positive
914,probably best cinematic depiction life manhatt...,positive
903,saw sweeney todd broadway starred george hearn...,positive
259,massive fan book orwell certainly favourite wr...,negative


In [41]:
df['sentiment'].value_counts()

sentiment
negative    264
positive    236
Name: count, dtype: int64

In [42]:
x=df['sentiment'].isin(['positive','negative'])
df=df[x]

In [43]:
df['sentiment']=df['sentiment'].map({'positive':1,'negative':0})
df.head()

,review,sentiment
323,hilarious insightful perspective dating world ...,1
439,story line game jerky appears game writer trie...,1
914,probably best cinematic depiction life manhatt...,1
903,saw sweeney todd broadway starred george hearn...,1
259,massive fan book orwell certainly favourite wr...,0


In [44]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [46]:
vectorizer=CountVectorizer(max_features=100)
X=vectorizer.fit_transform(df['review'])
y=df['sentiment']

In [47]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.25,random_state=42)

In [48]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/NitinNandeshwar/MLOPs_Project1.mlflow')
dagshub.init(repo_owner='NitinNandeshwar',repo_name='MLOPs_Project1',mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Suman\anaconda3\envs\mlops1\lib\site-packages\rich\live.py:229: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1dc423f5-1135-4e04-833e-89b2360f2159&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e436005a47489d89588779e53fc3b398dc50a4e22f155868ad94d64881dafd8a




Accessing as NitinNandeshwar

Initialized MLflow to track repo "NitinNandeshwar/MLOPs_Project1"

Repository NitinNandeshwar/MLOPs_Project1 initialized!

2025/06/28 15:54:46 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/62e8187d480a4b5e812534a8cf4f655d', creation_time=1751122486198, experiment_id='0', last_update_time=1751122486198, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score

# Configure Logging
logging.basicConfig(level=logging.INFO,format="%(asctime)s - %(levelname)s - %(message)s ")

logging.info("Starting MLflow run ...")

with mlflow.start_run():
    start_time=time.time()

    try:
        logging.info("Logging preprocessing parameters ...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features",100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initsializing Logistic Regression model ...")
        model= LogisticRegression(max_iter=1000) # Increase max_iter to prevent non-convergences issues

        logging.info("Fitting the model ...")
        model.fit(X_train,y_train)
        logging.info("Model training complete.")

        logging.info("Logging model paramters...")
        mlflow.log_param("model","Logistic Regression")

        logging.info("Making predictions ...")
        y_pred= model.predict(X_test)

        logging.info("Calculating evaluation metrics ...")
        accuracy=accuracy_score(y_test,y_pred)
        precision=precision_score(y_test,y_pred)
        recall=recall_score(y_test,y_pred)
        f1=f1_score(y_test,y_pred)

        logging.info("Logging evaluation metrics ...")
        mlflow.log_metric("accuracy",accuracy)
        mlflow.log_metric("precision",precision)
        mlflow.log_metric("recall",recall)
        mlflow.log_metric("f1_score",f1)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")


    except Exception as e:
        logging.error(f"An error occurred : {e}",exc_info=True)